In [6]:
from sklearn import svm
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import classification_report
from operator import itemgetter, attrgetter
import numpy as np
import nltk
import math
nltk.download(['stopwords','rslp','punkt'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/svncjus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /home/svncjus/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to /home/svncjus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
def SVM(features, labels, folds):
  print('Linear SVM start: ')
  clf = svm.SVC(kernel='linear', C=1)
  predict = cross_val_predict(clf, features, labels, cv=folds)
  cm = confusion_matrix(labels, predict)
  print(cm)
  print(classification_report(labels, predict, target_names=np.unique(labels)))
  print('----------------------------------------------')
  return predict, cm


In [8]:
def loadFile(filename, stopwords, steemer):
  f = open(filename, "r", encoding="utf-8-sig")
  data = []
  while True:
      chunk = f.readline() #pega linha do arquivo
      if chunk == '': #verifica se arquivo acabou
          break
      text = chunk.split(';;') #divide em classe (text[0]) e texto (text[1])
      
      tokens = nltk.word_tokenize(text[1]) #tokeniza a linha
      tokensLimpo = []
      for palavra in tokens:
          if (palavra not in stopwords):
              tokensLimpo.append(stemmer.stem(palavra)) #insere na lista de tokens limpos o steemer da palavra
      instance = (text[0],tokensLimpo) #cria uma instancia 
      data.append(instance)

  #-------------- BoW -------------------
  #Elenca tokens unicos
  allwords = []
  for text in data:
      for word in text[1]:
          if word not in allwords:
              allwords.append(word)  
  allwords.sort() #ordena tokens

  #gera Matriz de ocorrências de palavras
  qtdPalavras = len(allwords)
  features = []
  labels = []
  for texto in data:
      ocorrencias = []
      labels.append(texto[0])
      for palavra in allwords:
          if palavra in texto[1]:
              ocorrencias.append(texto[1].count(palavra))
          else:
              ocorrencias.append(0)
      features.append(ocorrencias)

  return labels, features, allwords

In [9]:
#Lista StopWords
stopwords = nltk.corpus.stopwords.words('portuguese') #carrega stopwords da lingua portuguesa disponíveis no NLTK
stopwords+= (',','.','(',')','"',"'",'´','`','!','$','%','&','...','-',':',';','?','``','\'\'') #acrescenta simbolos
stopwords+= ('a','e','i','o','u','A','E','I','O','U') #acrescenta também vogais
#Stemmer PTBR
stemmer = nltk.stem.RSLPStemmer()

In [10]:
InputFile = "2000_textos.txt"

In [53]:
labels7, features7, allwords7 = loadFile(InputFile, stopwords, stemmer)

In [7]:
print('----------------------------------------------') 
print("Instâncias 7 classes: "+str(len(labels7)))
print("Features 7 classes: "+str(len(features7[0])))

predict7, cm7 = SVM(features7, labels7, 10)

----------------------------------------------
Instâncias 7 classes: 2000
Features 7 classes: 6658
Linear SVM start: 
[[ 37  13   7  80   2  27  18]
 [  8 101  18  80   7  13  35]
 [  6  18 103  22   4  16  53]
 [ 41  57   8 389   1  21  25]
 [  3  13   8   7  22   7  23]
 [ 24  29  15  39   4 121  20]
 [ 18  41  43  34  16  24 279]]
              precision    recall  f1-score   support

     alegria       0.27      0.20      0.23       184
    desgosto       0.37      0.39      0.38       262
        medo       0.51      0.46      0.49       222
      neutro       0.60      0.72      0.65       542
       raiva       0.39      0.27      0.32        83
    surpresa       0.53      0.48      0.50       252
    tristeza       0.62      0.61      0.61       455

    accuracy                           0.53      2000
   macro avg       0.47      0.45      0.45      2000
weighted avg       0.52      0.53      0.52      2000

----------------------------------------------


In [12]:
labels7[:10]

['alegria',
 'tristeza',
 'surpresa',
 'tristeza',
 'neutro',
 'neutro',
 'tristeza',
 'raiva',
 'surpresa',
 'surpresa']

In [50]:
len(labels7)

2000

In [54]:
len(features7)

2000

In [59]:
def loadFile2(filename, stopwords, steemer):
  f = open(filename, "r", encoding="utf-8-sig")
  data = []
  while True:
      chunk = f.readline() #pega linha do arquivo
      if chunk == '': #verifica se arquivo acabou
          break
      text = chunk.split(';;') #divide em classe (text[0]) e texto (text[1])
      
      tokens = nltk.word_tokenize(text[1]) #tokeniza a linha
      tokensLimpo = []
      for palavra in tokens:
          if (palavra not in stopwords):
              tokensLimpo.append(stemmer.stem(palavra)) #insere na lista de tokens limpos o steemer da palavra
      instance = (text[0],tokensLimpo) #cria uma instancia 
      data.append(instance)

  #-------------- BoW -------------------
  #Elenca tokens unicos
  allwords = []
  for text in data:
      for word in text[1]:
          if word not in allwords:
              allwords.append(word)  
  allwords.sort() #ordena tokens

  #gera Matriz de ocorrências de palavras
  qtdPalavras = len(allwords)
  features = []
  labels = []
  for texto in data:
    ocorrencias = []
    #transformação das labels para redução de granularidade
    if texto[0]=="alegria":
        labels.append("positivo")
    elif texto[0] in ("raiva","medo","desgosto","tristeza"):
        labels.append("negativo")
    elif texto[0] == "surpresa":
        continue
    else:
        labels.append(texto[0])
    for palavra in allwords:
        #adição do condicional para não incluir instâncias classificadas como "surpresa"
        if palavra in texto[1] and texto[0] != "surpresa":
            ocorrencias.append(texto[1].count(palavra))
        else:
            ocorrencias.append(0)
    features.append(ocorrencias)

  return labels, features, allwords

In [60]:
labels3, features3, allwords3 = loadFile2(InputFile, stopwords, stemmer)

In [55]:
len(labels3)

1748

In [61]:
labels3[-25:]

['negativo',
 'negativo',
 'negativo',
 'positivo',
 'negativo',
 'negativo',
 'negativo',
 'negativo',
 'negativo',
 'negativo',
 'neutro',
 'negativo',
 'negativo',
 'neutro',
 'positivo',
 'negativo',
 'negativo',
 'neutro',
 'negativo',
 'negativo',
 'negativo',
 'negativo',
 'negativo',
 'positivo',
 'negativo']

In [56]:
len(features3)

1748

In [62]:
print('----------------------------------------------') 
print("Instâncias 3 classes: "+str(len(labels3)))
print("Features 3 classes: "+str(len(features3[0])))

predict3, cm3 = SVM(features3, labels3, 10)

----------------------------------------------
Instâncias 3 classes: 1748
Features 3 classes: 6658
Linear SVM start: 
[[880 125  17]
 [161 354  27]
 [ 64  74  46]]
              precision    recall  f1-score   support

    negativo       0.80      0.86      0.83      1022
      neutro       0.64      0.65      0.65       542
    positivo       0.51      0.25      0.34       184

    accuracy                           0.73      1748
   macro avg       0.65      0.59      0.60      1748
weighted avg       0.72      0.73      0.72      1748

----------------------------------------------


## Conclusões

De Acordo com o previsto, a exclusão da categoria "surpresa" e a redução da dimensionalidade das classes, provocou um aumento expressivo na performance do modelo preditivo conforme podemos perceber no relatório acima. A precisão avançou quase 30 p.p, recall e f1-score quase 20 p.p.